# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-09-13&until-occurred-date=2021-09-20


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

12792 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-09-13&until-occurred-date=2021-09-20
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=df405233-f398-4063-b960-2ec661523c4f&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-09-13&until-occurred-date=2021-09-20
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=f58d12c0-1a53-41bb-b5cf-d5c1f732030f&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-09-13&until-occurred-date=2021-09-20
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-07-26.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.13.452194...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.07.08.21260210...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255739...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.04.20.21255670...
REST API query complete  200
REST API query started for 10.1101/2021.06.20.21259209...
REST API query complete  200
REST API query started for 10.1101/2021.04.15.21252192...
REST API query complete  200
REST API query started for 10.1101/2021.08.17.21262169...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.06.11.21258690...
REST API query complete  200
REST API query started for 10.1101/2021.08.08

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.09.14.460138,39,bioRxiv,Biochemistry,False,Structure of a bacterial Rhs effector exported...,"[{'name': 'Patrick Günther'}, {'name': 'Dennis...",<p>The type VI <underline>s</underline>ecretio...,2021-09-14
1,10.1101/2021.09.13.460111,37,bioRxiv,Microbiology,True,Therapeutic efficacy of an oral nucleoside ana...,"[{'name': 'Alexandra Schafer'}, {'name': 'Davi...",<p>The COVID-19 pandemic remains uncontrolled ...,2021-09-13
2,10.1101/2021.09.09.21263331,34,medRxiv,Epidemiology,True,"Second wave of the Covid-19 pandemic in Delhi,...","[{'name': 'Nandini Sharma'}, {'name': 'Pragya ...",<sec><title>Background</title><p>We report the...,2021-09-13
3,10.1101/2021.09.13.460168,32,bioRxiv,Bioinformatics,False,Random Forest Factorization Reveals Latent Str...,"[{'name': 'Boris M. Brenerman'}, {'name': 'Ben...",<p>Single-cell RNA sequencing data contain pat...,2021-09-14
4,10.1101/2021.09.01.21262607,31,medRxiv,Health Informatics,False,ADataViewer: Exploring Semantically Harmonized...,"[{'name': 'Yasamin Salimi'}, {'name': 'Daniel ...","<sec><title>INTRODUCTION</title><p>Currently, ...",2021-09-14
5,10.1101/2021.09.13.21263499,30,medRxiv,Epidemiology,False,The natural history of TB disease-a synthesis ...,"[{'name': 'Alexandra S Richards'}, {'name': 'B...",<sec><title>Background</title><p>Prevalence su...,2021-09-16
6,10.1101/2021.09.15.460550,23,bioRxiv,Neuroscience,False,Intact reading ability in spite of a spatially...,"[{'name': 'Jin Li'}, {'name': 'Evelina Fedoren...",<p>The visual word form area (VWFA) is an expe...,2021-09-16
7,10.1101/2021.09.15.460516,21,bioRxiv,Neuroscience,False,Long-term Motor Learning in the Wild with High...,"[{'name': 'Jennifer B Listman'}, {'name': 'Jon...",<p>Motor learning occurs over long periods of ...,2021-09-17
8,10.1101/2021.09.14.460331,16,bioRxiv,Immunology,False,Defective endosome-TGN retrograde transport pr...,"[{'name': 'Zhirong Zhang'}, {'name': 'Li Ran'}...",<p>Inflammasome complexes are pivotal in the i...,2021-09-16
9,10.1101/2021.09.14.460408,15,bioRxiv,Systems Biology,True,Synthetic lethality-based prediction of anti-S...,"[{'name': 'Lipika R. Pal'}, {'name': 'Kuoyuan ...",<p>Novel strategies are needed to identify dru...,2021-09-15


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

63 preprints (including 11 covering SARS-CoV-2, 54 from bioRxiv and 9 from medRxiv) published in the last 7 days before 2021-09-20 had been tweeted at least 3 times (maximum 39).